In [1]:
from src.game.game_model import GameModel
from src.game.game_tree import GameTree

from algorithms.minmax import MinMax

# Define tic tac toe game

In [2]:
gm = GameModel( 
agents_number=2, default_agent_features=['not starter', 'X'], additional_agent_features=[['starter'], ['O']], 
agent_features_descriptions="2 players with feature 1 indicating who is starting, and feature 2 indicating their symbol.",
game_name="tic-tac-toe")
gm.add_action_space("board", dimensions=[3, 3], default_labels=['free'], additional_labels=[['X', 'O']], dimensions_descriptions="3x3 board.")

# Disable actions on the agent feature space.
gm.disable_actions(on="agent")
gm.agents[1, 1] = 'O'
gm.agents[1, 0] = 'starter'

def tic_tac_toe_endgame(game):
    board = game.action_spaces["board"]
    # Check rows for winning condition
    for row in board:
        if row[0] == row[1] == row[2] != 'free':
            return True

    # Check columns for winning condition
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] != 'free':
            return True

    # Check diagonals for winning condition
    if board[0][0] == board[1][1] == board[2][2] != 'free':
        return True
    if board[0][2] == board[1][1] == board[2][0] != 'free':
        return True

    # If no winner, return False
    return False

gm.set_endgame(tic_tac_toe_endgame)

gm.action_is_violation_if(lambda who, where, what, game: not game.started and game.agents[who][0] != 'starter', rule_description="This is not the starting player and this is the first turn.")
gm.action_is_violation_if(lambda who, where, what, game: game.started and who == game.actions[-1]['who'], rule_description="Players cannot play two times consecutively")
gm.action_is_violation_if(lambda who, where, what, game: where != 'free', "board", rule_description="The space needs to be free to put a sign on it.")
gm.action_is_violation_if(lambda who, where, what, game: game.agents[who][1] != what, rule_description="Agents can only put their own sign.")


        You can add spaces on which to perform actions with the function add_action_space(dimensions, default_labels, additional_labels, dimensions_description).
        For example most games need a board. By default there is the agent space, allowing for actions onto the agents features.
        You can disable actions on an action space with 

        You can use the method gm.action_is_violation_if(rule, rule_description) to express rules for the game.

        Use help(gm.action_is_violation_if) for help on how to define rules.
        The rule: "Nothing is allowed if the game is ended." is defined by default, to delete it use gm.delete_rule(1).
        Use gm.print_rules() to see all the rules that have been setted.
        
        Use gm.set_endgame(callable_function(game)) to define based on what dynamics your game should end.
        The callable function should have one only input parameter, which represents the game itself, 
        which can be used to refer to anything i

## Define the game tree

In [4]:
gmt = GameTree(gm, "board")

# Define the MinMax

## Scoring function:

In [3]:
import numpy as np
def scoring_function(state):
    """ Evaluate the Tic Tac Toe board state for the 'X' player's perspective """
    score = 0
    
    # Possible lines to check (3 rows, 3 columns, 2 diagonals)
    lines = []
    # Rows and columns
    for i in range(3):
        lines.append(state[i, :])  # Row
        lines.append(state[:, i])  # Column
    # Diagonals
    lines.append(np.array([state[i, i] for i in range(3)]))  # Main diagonal
    lines.append(np.array([state[i, 2 - i] for i in range(3)]))  # Anti-diagonal

    for line in lines:
        if np.all(line == "X"):
            score += 100  # 'X' wins
        elif np.all(line == "O"):
            score -= 100  # 'O' wins
        elif np.count_nonzero(line == "X") == 2 and np.count_nonzero(line == "free") == 1:
            score += 10  # 'X' is one move away from winning
        elif np.count_nonzero(line == "O") == 2 and np.count_nonzero(line == "free") == 1:
            score -= 10  # 'O' is one move away from winning

    return score

In [5]:
search = MinMax(gmt, scoring_function)

In [6]:
search.run(gmt.root.id)

('0_0', 0)